# Eine Day-ahead Prognose für den deutschen Strommarkt
### Mittels Zeitreihen

Import Python packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from tensorflow.keras import Sequential
from keras.layers import Dense,LSTM,Dropout, GRU, Activation, SimpleRNN
from keras.optimizers import RMSprop
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math as m
from tensorflow import keras
from keras.callbacks import EarlyStopping
from pytictoc import TicToc
%matplotlib inline


Import Smard Daten

In [ ]:
df= pd.read_csv("price_data.csv",sep=",",index_col=0)
df.index = pd.to_datetime(df.index)

Erstellung von Funktionen

In [ ]:
def split_dataset(length, dataset):
    "Aufteilung Dataset in Train und Test"
    train,test=dataset[:length],dataset[length:]
    return(train, test)


def dataset_preparing(inputs,outputs, dataset):
    " Teilt das Dataset in Inputvektoren und Outputvektoren ein"
    x_i=[]
    y_i=[]
    for i in range(inputs,len(dataset)-outputs,outputs):
        x_i.append(dataset[i-inputs:i])
        y_i.append(dataset[i:i+outputs])
    return np.array(x_i), np.array(y_i)


Daten skalieren

In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))
df=scaler.fit_transform(df.values)

Datenset in Trainings und Test daten unterteilen

In [ ]:
train=split_dataset(36000,df)[0]
test=split_dataset(36000,df)[1]

Trainings- und Testdaten in Input/Outputvekoren einteilen

In [ ]:
X_train= dataset_preparing(48,24,train)[0]
Y_train=dataset_preparing(48,24,train)[1]
X_test=dataset_preparing(48,24,test)[0]
Y_test=dataset_preparing(48,24,test)[1]

Shape der Vektoren in 3 Dimensionen ändern

In [ ]:
Y_train = np.reshape(Y_train, (Y_train.shape[0], Y_train.shape[1]))
Y_test=np.reshape(Y_test,(Y_test.shape[0],Y_test.shape[1]))

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))

## Neuronale Netze mit nur einer Schicht

RNN:

In [ ]:
model_RNN1 =  Sequential()
model_RNN1.add(SimpleRNN(units=24,input_shape=(X_train.shape[1], X_train.shape[2]),return_sequences=False,activation= "relu")) 
optimizer = RMSprop(lr=0.0001)
model_RNN1.compile(loss='mae', optimizer=optimizer, metrics=['mean_squared_error'])

es = EarlyStopping(monitor='val_loss', mode='min',patience=10,restore_best_weights=True)

history_RNN1 = model_RNN1.fit(X_train, Y_train, epochs=200,batch_size=220, validation_data=(X_test,Y_test),shuffle=False,callbacks=[es])
model_RNN1.summary() 


LSTM:

In [ ]:
model1 =  Sequential()
model1.add(LSTM(units=24,input_shape=(X_train.shape[1], X_train.shape[2]),return_sequences=False,activation= "relu"))  

optimizer = RMSprop(lr=0.0001)
model1.compile( optimizer=optimizer, loss='mae',metrics=['mean_squared_error'])

es = EarlyStopping(monitor='val_loss', mode='min',patience=10,restore_best_weights=True)

history_lstm1 = model1.fit(X_train, Y_train, epochs=200,batch_size=220, validation_data=(X_test,Y_test),shuffle=False,callbacks=[es])
model1.summary() 

GRU:

In [ ]:
model_GRU1 =  Sequential()
model_GRU1.add(GRU(units=24,input_shape=(X_train.shape[1], X_train.shape[2]),return_sequences=False,activation="relu"))
optimizer = RMSprop(lr=0.0001)
model_GRU1.compile(loss='mae', optimizer=optimizer, metrics=['mean_squared_error'])

es = EarlyStopping(monitor='val_loss', mode='min',patience=10,restore_best_weights=True)

history_GRU1 = model_GRU1.fit(X_train, Y_train, epochs=200, batch_size=220,validation_data=(X_test,Y_test),shuffle=False,callbacks=[es])

model_GRU1.summary() 


### Prognosen der einfachen Netze erstellen

In [ ]:
predict_RNN1 = model_RNN1.predict(X_test)
predict_RNN1 = scaler.inverse_transform(predict_RNN1)

predict_lstm1 = model1.predict(X_test)
predict_lstm1 = scaler.inverse_transform(predict_lstm1)

test_predict_GRU1 = model_GRU1.predict(X_test)
test_predict_GRU1 = scaler.inverse_transform(test_predict_GRU1)


## Neuronale Netze mit vier Schichten

RNN:

In [ ]:
model_RNN =  Sequential()
model_RNN.add(SimpleRNN(units=220,input_shape=(X_train.shape[1], X_train.shape[2]),return_sequences=True,activation= "relu")) 
model_RNN.add(Dropout(0.5))
model_RNN.add(SimpleRNN(units=140,return_sequences=True,activation= "relu")) 
model_RNN.add(Dropout(0.5))
model_RNN.add(SimpleRNN(units=80,return_sequences=True,activation= "relu"))
model_RNN.add(Dropout(0.3))
model_RNN.add(SimpleRNN(24, activation= "relu"))  
model_RNN.compile(loss='mae', optimizer='adam', metrics=['mean_squared_error'])

es = EarlyStopping(monitor='val_loss', mode='min',patience=10,restore_best_weights=True)

history_RNN = model_RNN.fit(X_train, Y_train, epochs=200,batch_size=220, validation_data=(X_test,Y_test),shuffle=False,callbacks=[es])
model_RNN.summary() 


LSTM:

In [ ]:
model =  Sequential()
model.add(LSTM(units=220,input_shape=(X_train.shape[1], X_train.shape[2]),return_sequences=True,activation= "relu")) 
model.add(Dropout(0.5))
model.add(LSTM(units=140,return_sequences=True,activation= "relu")) 
model.add(Dropout(0.5))
model.add(LSTM(units=80,return_sequences=True,activation= "relu"))
model.add(Dropout(0.3))
model.add(LSTM(24, activation= "relu"))  
model.compile(loss='mae', optimizer='adam', metrics=['mean_squared_error'])

es = EarlyStopping(monitor='val_loss', mode='min',patience=10,restore_best_weights=True)

history_lstm = model.fit(X_train, Y_train, epochs=200,batch_size=220, validation_data=(X_test,Y_test),shuffle=False,callbacks=[es])
model.summary() 


GRU:

In [ ]:
model_GRU =  Sequential()
model_GRU.add(GRU(units=220,input_shape=(X_train.shape[1], X_train.shape[2]),return_sequences=True,activation="relu")) 
model_GRU.add(Dropout(0.5))
model_GRU.add(GRU(units=140,return_sequences=True,activation="relu"))
model_GRU.add(Dropout(0.5))
model_GRU.add(GRU(units=80,return_sequences=True,activation="relu"))
model_GRU.add(Dropout(0.3))
model_GRU.add(GRU(24,activation="relu"))
model_GRU.compile(loss='mae', optimizer='adam', metrics=['mean_squared_error'])

es = EarlyStopping(monitor='val_loss', mode='min',patience=10,restore_best_weights=True)

history_GRU = model_GRU.fit(X_train, Y_train, epochs=200, batch_size=220,validation_data=(X_test,Y_test),shuffle=False,callbacks=[es])
model_GRU.summary() 


### Fehlerfunktionen anzeigen lassen

In [ ]:
plt.figure(figsize=(10,6))
plt.title("Trainings und Test Fehler")
plt.xlabel("Epoche")
plt.plot(history_GRU.history["loss"])
plt.plot(history_GRU.history["val_loss"])
plt.plot(history_RNN.history["loss"])
plt.plot(history_RNN.history["val_loss"])
plt.plot(history_lstm.history["loss"])
plt.plot(history_lstm.history["val_loss"])
plt.legend(["GRU Train loss","GRU Test loss", "RNN Train loss","RNN Test loss","LSTM train loss", "LSTM Test loss"])
plt.show()

### Prognosen erstellen

In [ ]:
predict_RNN = model_RNN.predict(X_test)
predict_RNN = scaler.inverse_transform(predict_RNN)

predict_lstm = model.predict(X_test)
predict_lstm = scaler.inverse_transform(predict_lstm)

test_predict_GRU = model_GRU.predict(X_test)
test_predict_GRU = scaler.inverse_transform(test_predict_GRU)


Zurück skalierung der Test Outputwerte

In [ ]:
Y_test = scaler.inverse_transform(Y_test)

### Modellgüte (MAE) berechnen 

In [ ]:
print("Multi Layer LSTM hat ein MAE von {}".format(round(mean_absolute_error(Y_test,predict_lstm),2)))
print("Multi Layer GRU hat ein MAE von {}".format(round(mean_absolute_error(Y_test,test_predict_GRU),2)))
print("Multi Layer RNN hat ein MAE von {}".format(round(mean_absolute_error(Y_test,predict_RNN),2)))
print("Single Layer LSTM hat ein MAE von {}".format(round(mean_absolute_error(Y_test,predict_lstm1),2)))
print("Single Layer GRU hat ein MAE von {}".format(round(mean_absolute_error(Y_test,test_predict_GRU1),2)))
print("Single Layer RNN hat ein MAE von {}".format(round(mean_absolute_error(Y_test,predict_RNN1),2)))

### Ergebnisse anzeigen lassen

In [ ]:
plt.figure(figsize=(10,4))
plt.title("LSTM Model 2020-06-29 MAE: {}".format(round(mean_absolute_error(Y_test[-1],predict_lstm[-1])),2))
plt.ylabel('Electricity price', size=15)
plt.xlabel('Time', size=15)
plt.plot(Y_test[:][-1],marker="x")
plt.plot(predict_lstm[:][-1], marker="x")
plt.legend(["Test Price","Predictions LSTM"],loc="best",prop={'size': 12})
plt.show();

In [ ]:
plt.figure(figsize=(10,4))
plt.title("GRU Model 2020-06-29 MAE: {}".format(round(mean_absolute_error(Y_test[-1],test_predict_GRU[-1])),2))
plt.ylabel('Electricity price', size=15)
plt.xlabel('Time', size=15)
plt.plot(Y_test[:][-1],marker="x")
plt.plot(test_predict_GRU[:][-1], marker="x")
plt.legend(["Test Price","Predictions GRU"],loc="best",prop={'size': 12})
plt.show();